In [ ]:
import os,re, pickle, random
import numpy as np
import pandas as pd
from itertools import chain
from bs4 import BeautifulSoup
from xgboost import XGBRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import keras
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import MeanAbsoluteError

import matplotlib
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, roc_curve, auc
matplotlib.rc('xtick', labelsize=16) 
matplotlib.rc('ytick', labelsize=16) 

In [ ]:
# Go to that directory where log files (single stuck at fault manifest as Missing/Delayed and Spurious) are saved
cd 128x128/ 

In [ ]:
def getInfo(file, ip_spks):

    """
    Args:
    file : Log files 
    ip_spks : Number of input neurons or pre-synaptic neurons

    return :
    df : Dataframe
    
    """

    with open(file) as f:
        soup = BeautifulSoup(f.read(), "html.parser")

    df = pd.DataFrame()

    # Making column names
    cols = ["Time(s)"]
    for i in range(ip_spks):
      cols.append("spk_ip_" + str(i))
      cols.append("#Spikes_" + str(i))
      cols.append("ISI_" + str(i))

    for i in range(ip_spks):
      cols += ["spk_out_" + str(i), "#Spikes_out_"+str(i), "ISI_out_"+str(i)]

    time = [int(i.split(":")[1]) for i in re.findall(r"Time : \d+", soup.text)]

    df["Time(s)"] = time

    # NeuronID info for each spike generating neuron
    n_id_info = re.findall("NeuronID: (.+)", soup.text)

    spikes_info = [list(map(int,re.findall(r'(\d+)', i))) for i in n_id_info]
    
    n_id_info_v1 = []

    for idx in range(0, len(spikes_info), ip_spks):
      var = list(chain.from_iterable((x[0],len(x)-1, x[1:]) for x in spikes_info[idx: idx + ip_spks]))
      n_id_info_v1.append(var)
  
    df[cols[1:-3 *ip_spks]] = n_id_info_v1

    # Output column info
    out = re.findall("Output Neuron at column (.+)", soup.text)

    spikes_info_out = [list(map(int, re.findall(r'(\d+)', i))) for i in out]
    out_info = []

    for idx in range(0, len(spikes_info_out), ip_spks):
      var1 = list(chain.from_iterable((x[0],len(x)-1, x[1:]) for x in spikes_info_out[idx: idx + ip_spks]))
      out_info.append(var1)

    # Checking #instances
    assert len(out_info) == len(time) == len(n_id_info_v1)

    df[cols[-3*ip_spks:]] = out_info


    return df

In [ ]:
# To compute average ISI (Inter-spiking interval)
def getAvgIsi(x):

  if len(x) == 0:
    return 0

  elif len(x) == 1:
    return x[0]%1000

  else:
    x = [i%1000 for i in x]
    if x[0] == 0:
      x[0] = 1
    return sum([(x[i+1])- (x[i]) for i in range(len(x)-1)])/(len(x)-1)

In [ ]:
# To compute COV (coefficient of variation) or Variance
def cov_or_variance(x, type_metric = "cov"):

  if len(x) == 0:
    return 0

  elif len(x) == 1:
    return x[0]%1000

  else:
    x = [i%1000 for i in x]
    if x[0] == 0:
      x[0] = 1
    imd = np.array([x[i]%1000 for i in range(len(x))])
    mu = np.mean(imd)
    
    if type_metric == "cov": 
      # coefficient of variation
      cov = np.sqrt(np.sum(np.square(imd - mu))/ (len(imd) - 1))/mu
      return cov
    else:
      # Variance of spikes
      var_spk = np.sum(np.square(imd - mu))/ (len(imd) - 1)
      return var_spk

In [ ]:
# # To make a directory to save all log files (Missing/Delayed and Spurious) with different utilizations for each pre-synaptic neuron or input neuron
utilization = range(10,51,10) 

for ut in utilization:
  os.mkdir("128x128/"+ str(ut) + "_percent/files_128_raw")

In [ ]:
def SaveRawFile(dataframe, fol, savePath, cp):
  """
  Args :
  dataframe: Dataframe
  fol : File name of post-synaptic neuron ID
  savePath : The path to save .xlsx file 
  cp :  Crosspoint
  
  """

  columns = dataframe.columns
  f = pd.DataFrame()

  idx = 0
  for e in columns:
    
    if e.startswith("ISI_"):

      if e.startswith("ISI_out_"):
          f[e] =  dataframe[e]
          f['Avg_ISI_out_'+e.split("ISI_out_")[-1]] =  dataframe[e].apply(lambda x: getAvgIsi(x))
          f['Cov_spk_out_'+e.split("ISI_out_")[-1]] =  dataframe[e].apply(lambda x: cov_or_variance(x, type_metric = "cov"))
          f['Var_spk_out_'+e.split("ISI_out_")[-1]] =  dataframe[e].apply(lambda x: cov_or_variance(x, type_metric = "var"))
        
      else:
        f[e] =  dataframe[e]
        f['Avg_ISI_'+str(idx)] =  dataframe['ISI_' + str(idx)].apply(lambda x: getAvgIsi(x))
        f['Cov_spk_ip_'+str(idx)] =  dataframe['ISI_' + str(idx)].apply(lambda x: cov_or_variance(x, type_metric = "cov"))
        f['Var_spk_ip_'+str(idx)] =  dataframe['ISI_' + str(idx)].apply(lambda x: cov_or_variance(x, type_metric = "var"))
        idx += 1

    else:
      f[e] = dataframe[e]

  f.to_excel(savePath + fol + "_" + str(cp) + ".xlsx", index = False)
  

# Saving log files in the desirable and readable format

In [1]:
"""
cp : Crosspoints
util : Utilization (10% or 20% or 30% or 40% or 50% of the crossbar array (in terms of the crosspoints))
log_path : The path from where the log file will be read.
save_path : The path where the log file will be saved.

"""

cp_util = [(13,10), (26,20), (39,30), (52,40), (64,50)]
type_fol = ["Output_Missing", "Output_Spurious"] 

for params in cp_util:
  cp,util = params

  for fol in type_fol:
    logs_path = "128x128/" + str(util) + "_percent/" + fol + "/log_weight.txt"
    savePath = "128x128/" + str(util) + "_percent/" + "files_128_raw/"

    ## Saving raw data in excel file
    print(f"{util}% utilization ({cp} crosspoints) for {fol}.")
    df = getInfo(logs_path, 128)
    SaveRawFile(df, fol, savePath, cp)


10% utilization (13 crosspoints) for Output_Missing.
10% utilization (13 crosspoints) for Output_Spurious.
20% utilization (26 crosspoints) for Output_Missing.
20% utilization (26 crosspoints) for Output_Spurious.
30% utilization (39 crosspoints) for Output_Missing.
30% utilization (39 crosspoints) for Output_Spurious.
40% utilization (52 crosspoints) for Output_Missing.
40% utilization (52 crosspoints) for Output_Spurious.
50% utilization (64 crosspoints) for Output_Missing.
50% utilization (64 crosspoints) for Output_Spurious.


In [ ]:
# Sort the name of file of post-synaptic neuron ID

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):

    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

In [ ]:
# # To create a directory to save all features (Spike rate, Avg. ISI/COV/Variance) and their ground truth (Observed ISI) 
utilization = range(10,51,10) 
crossbarName = "128x128"

for ut in utilization:
  os.mkdir(crossbarName + "/"+ str(ut) + "_percent/StackedFiles_avg_ISI")
  # os.mkdir(crossbarName + "/"+ str(ut) + "_percent/StackedFiles_cov_spk")
  # os.mkdir(crossbarName + "/"+ str(ut) + "_percent/StackedFiles_var_spk")

In [ ]:
type_fol = ["Output_Missing", "Output_Spurious"]

for ut in utilization:
  for fol in type_fol:
    os.mkdir(crossbarName + "/" + str(ut) + "_percent/StackedFiles_avg_ISI/"+fol)
    # os.mkdir(crossbarName + "/" + str(ut) + "_percent/StackedFiles_cov_spk/"+fol)
    # os.mkdir(crossbarName + "/" + str(ut) + "_percent/StackedFiles_var_spk/"+fol)

In [ ]:
"""
Extracting features (Spike rate, Avg. ISI/COV/Variance) and their ground truth (Observed ISI) and then
save in .npy format.

"""

def StackedFile(path, fileName, nG, savePath, variance_feat = False):
    """
    Args: 
    path : The path to read excel files
    nG : Number of post-synaptic neurons
    variance_feat : Avg. ISI/COV/Variance as features included or not
    fileName : An excel file (Output_Spurious, Output_Missing)
    savePath : The path where the excel files will be saved in a numpy format in a parsed version based on cp.

    """

    """
    Predicting only average ISI_out, not #Spikes_out
    """
    

    # Number of crosspoints (numCp)
    numCp = int(fileName.split("_")[-1][:-5]) 
    print(f"File Name : {fileName} | Number of crosspoints : {numCp}")

    file = pd.read_excel(path + fileName)

    cols_drop = []

    for id in range(nG):
      cols_drop.append("spk_ip_" + str(id))
      cols_drop.append("spk_out_" + str(id))
      cols_drop.append("ISI_" + str(id))
      cols_drop.append("ISI_out_" + str(id))
      cols_drop.append("#Spikes_out_" + str(id))

      if variance_feat == "var":
        cols_drop.append("Cov_spk_ip_" + str(id))
        cols_drop.append("Cov_spk_out_" + str(id))
        cols_drop.append("Avg_ISI_" + str(id))
        cols_drop.append("Avg_ISI_out_" + str(id))

      elif not variance_feat: 
        cols_drop.append("Cov_spk_ip_" + str(id))
        cols_drop.append("Cov_spk_out_" + str(id))
        cols_drop.append("Var_spk_ip_" + str(id))
        cols_drop.append("Var_spk_out_" + str(id))
      else:
        cols_drop.append("Avg_ISI_" + str(id))
        cols_drop.append("Avg_ISI_out_" + str(id))
        cols_drop.append("Var_spk_ip_" + str(id))
        cols_drop.append("Var_spk_out_" + str(id))
        

    cols_drop.extend(['Time(s)'])

    file.drop(cols_drop, axis=1, inplace=True)

    file_v1 = file.reset_index(drop=True)

    if variance_feat == "var":
        y_cols = ["Var_spk_out_" + str(idx) for idx in range(nG)]
    elif not variance_feat: 
        y_cols = ["Avg_ISI_out_" + str(idx) for idx in range(nG)]
    else:
        y_cols = ["Cov_spk_out_" + str(idx) for idx in range(nG)]
        

    X_train  = file_v1.drop(y_cols, axis=1).values
    y_train =  file_v1[y_cols].values
    
    X_train_v1 = np.concatenate([X_train[:, :numCp*2], np.zeros((X_train.shape[0], X_train.shape[1] - (numCp*2)))], axis =1)

    
    X_tr, y_tr =  X_train_v1, y_train

      
    with open(savePath + "/X_train_stacked.npy", 'wb') as f:
      np.save(f, X_tr)

    for col in range(nG):
      with open(savePath +"/y_train_stacked_" + str(col) + ".npy", 'wb') as f:
        np.save(f, y_tr[:,col])


In [ ]:
def run(utilization, variance_feat, nG, crossbarName):

  for ut in utilization:
    type_files = [i for i in os.listdir(crossbarName + "/"+ str(ut) + "_percent/files_"+str(nG)+"_raw/") if i.endswith("xlsx")]
    for file in type_files:
      fol = "_".join(file.split("_")[:-1])
      if variance_feat == "var":
        savePath = crossbarName + "/"+ str(ut) + "_percent/StackedFiles_var_spk/"+fol
      elif not variance_feat:
        savePath = crossbarName + "/"+ str(ut) + "_percent/StackedFiles_avg_ISI/"+fol
      else:
        savePath = crossbarName + "/"+ str(ut) + "_percent/StackedFiles_var_spk/"+fol

      StackedFile(crossbarName + "/"+ str(ut) + "_percent/files_"+str(nG)+"_raw/", file, nG, savePath, variance_feat = variance_feat)

In [2]:
# Considering #Spikes & Avg ISI as features

variance_feat = False # avg_ISI
crossbarName = "128x128"
utilization = range(10,51,10)
nG = 128

run(utilization, variance_feat, nG, crossbarName)

File Name : Output_Missing_13.xlsx | Number of crosspoints : 13
File Name : Output_Spurious_13.xlsx | Number of crosspoints : 13
File Name : Output_Missing_26.xlsx | Number of crosspoints : 26
File Name : Output_Spurious_26.xlsx | Number of crosspoints : 26
File Name : Output_Missing_39.xlsx | Number of crosspoints : 39
File Name : Output_Spurious_39.xlsx | Number of crosspoints : 39
File Name : Output_Missing_52.xlsx | Number of crosspoints : 52
File Name : Output_Spurious_52.xlsx | Number of crosspoints : 52
File Name : Output_Missing_64.xlsx | Number of crosspoints : 64
File Name : Output_Spurious_64.xlsx | Number of crosspoints : 64


In [3]:
# Considering #Spikes & Variance as features

# variance_feat = "var" # Variance
# crossbarName = "128x128"
# utilization = range(10,51,10)
# nG = 128

# run(utilization, variance_feat, nG, crossbarName)

File Name : Output_Missing_13.xlsx | Number of crosspoints : 13
File Name : Output_Spurious_13.xlsx | Number of crosspoints : 13
File Name : Output_Missing_26.xlsx | Number of crosspoints : 26
File Name : Output_Spurious_26.xlsx | Number of crosspoints : 26
File Name : Output_Missing_39.xlsx | Number of crosspoints : 39
File Name : Output_Spurious_39.xlsx | Number of crosspoints : 39
File Name : Output_Missing_52.xlsx | Number of crosspoints : 52
File Name : Output_Spurious_52.xlsx | Number of crosspoints : 52
File Name : Output_Missing_64.xlsx | Number of crosspoints : 64
File Name : Output_Spurious_64.xlsx | Number of crosspoints : 64


In [4]:
# Considering #Spikes & COV as features

# variance_feat = True # COV
# crossbarName = "128x128"
# utilization = range(10,51,10)
# nG = 128
# run(utilization, variance_feat, nG, crossbarName)

File Name : Output_Missing_13.xlsx | Number of crosspoints : 13
File Name : Output_Spurious_13.xlsx | Number of crosspoints : 13
File Name : Output_Missing_26.xlsx | Number of crosspoints : 26
File Name : Output_Spurious_26.xlsx | Number of crosspoints : 26
File Name : Output_Missing_39.xlsx | Number of crosspoints : 39
File Name : Output_Spurious_39.xlsx | Number of crosspoints : 39
File Name : Output_Missing_52.xlsx | Number of crosspoints : 52
File Name : Output_Spurious_52.xlsx | Number of crosspoints : 52
File Name : Output_Missing_64.xlsx | Number of crosspoints : 64
File Name : Output_Spurious_64.xlsx | Number of crosspoints : 64


In [ ]:
def getBinary(y_true, y_pred, margin):
  
  """
  It will binarize the continuous values which are y_true and y_pred with
  the help of margin and threshold to plot ROC (Receiver operating characteristic).

  Args:
  y_true : Observed avg. ISI (continuous values)
  y_pred : Predicted avg. ISI (continuous values)
  margin : The margin is the obtained optimal MAE or a user can take loose margin as per his choice to incur more false alaram or false fault

  return:
  y_t : Binarized y_true
  y_p : Binarized y_pred

  The reason to opt for ROC curve because it offers an elegant way to
  plot true fault detection rate versus false fault detection rate.
 
  """

  thresh = pd.Series(y_true).median()   # We have taken median as a threshold because it is not impacted by the outliers
  y_t = []
  y_p = []

  for t,p in zip(y_true, y_pred):
    if np.abs(t - p) <= margin:
      if t> thresh:
        y_t.append(1)
        y_p.append(1)
      else:
        y_t.append(0)
        y_p.append(0)

    else:
      if (t > thresh) and (p > thresh):  # ex: t=60, p=80 but thresh = 55 --> t,p -> 1, therefore one of them has to be opposite to another because abs(t-p)>15.
        y_t.append(1)
        y_p.append(0)

      elif (t > thresh) and (p <= thresh):  # ex: t=60, p=40 but thresh = 55 --> t->1,  p-> 0, which satisfies the condition, one has to be opposite of another.
        y_t.append(1)
        y_p.append(0)

      elif (t <= thresh) and (p > thresh):  # ex: t=30, p=57 but thresh = 55 --> t->0,  p-> 1,
        y_t.append(0)
        y_p.append(1)

      elif (t <= thresh) and (p <= thresh): # ex: t=25, p=45 but thresh = 55 --> t->0, p-> 1, therefore one of them has to be opposite to another.
        y_t.append(0)
        y_p.append(1)
      

  y_t = np.array(y_t)
  y_p = np.array(y_p)

  return y_t, y_p

In [ ]:
def plotROC(y_true, y_pred, reg, margin, color):

  """
  To plot ROC

  Args: 
  y_true : Observed avg. ISI
  y_pred : Predicted avg. ISI
  reg :  Regressor Name
  margin : The margin is the obtained optimal MAE or a user can take loose margin as per his choice to incur more false alaram or false fault
  color : Color

  return:
  y_t : Binarized y_true
  y_p : Binarized y_pred

  """

  y_t, y_p = getBinary(y_true, y_pred, margin)
  
  fpr, tpr, _ = roc_curve(y_t, y_p )
  roc_auc = auc(fpr, tpr)

  print()
  print("False positive rate : ", fpr[1])
  print("True positive rate : ", tpr[1])
  print("ROC Area under curve : ", roc_auc)

  plt.figure(figsize = (6,6))
  lw = 2
  plt.plot(fpr, tpr, color= color, lw=lw, label=f"ROC curve (area = %0.2f) for {reg}." % roc_auc,linewidth=4)
  plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel("False Positive Rate", fontsize=15)
  plt.ylabel("True Positive Rate", fontsize=15)
  plt.title("Receiver operating characteristic example")
  plt.legend(loc="lower right")
  plt.show()

  return y_t, y_p

In [ ]:
def getPrediction(nG, ip_shape, scaling_mode, file_Type, crossbarName, model_type, model_name, optimal_maes, models_list, variance_feat =False):

  """
  Args:
  nG : Number of post-synaptic neurons
  scaling_mode : Type of scaling technique
  file_type : "Missing/Delayed" file or "Spurious" file
  crossbarName : Crossbar type 32x32 or 64x64 or 128x128
  model_type : "Tree" based or "NN" based
  model_name : which model to run
  models_list : List of model names
  optimal_maes : Obtained optimal MAEs for all columns
  ip_shape : Dimension of an example (Mainly for CNN)
  variance_feat : Coefficient of variation/Variance/Avg ISI as features included or not

  print : 
  Fault detection rate

  """

  row, col, chn = ip_shape
  utilization = range(10,51,10)

  print("*"*25 + " "+ file_Type.split("_")[-1] +" Spikes " + "*"*25)

  for util in utilization:
    print("*"*25 + " Utilization : " + str(util) + "% " + "*"*25)
    print()
    print("Scaling Mode : ", scaling_mode)
    print("Model Name : ", model_name, "\n")

    if variance_feat == "var":
      with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_var_spk/" + file_Type + "/X_train_stacked.npy", 'rb') as f:
        X = np.load(f)
    elif not variance_feat:
      with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_avg_ISI/" + file_Type + "/X_train_stacked.npy", 'rb') as f:
        X = np.load(f)
    else:
      with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_cov_spk/" + file_Type + "/X_train_stacked.npy", 'rb') as f:
        X = np.load(f)

    prediction_wrt_column = []

    for id in range(nG):

      if variance_feat == "var":
        with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_var_spk/" + file_Type +"/y_train_stacked_" + str(id) + ".npy", 'rb') as f:
          y = np.load(f)
          y = y/10000  # (for better fit)
      elif not variance_feat:
        with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_avg_ISI/" + file_Type +"/y_train_stacked_" + str(id) + ".npy", 'rb') as f:
          y = np.load(f)
          y = y/10  # (for better fit)
      else:
        with open(crossbarName+ "/"+ str(util) + "_percent/StackedFiles_cov_spk/" + file_Type +"/y_train_stacked_" + str(id) + ".npy", 'rb') as f:
          y = np.load(f)
      
      X_test, y_test = X[:5000], y[:5000] 

      # Standard Scaling
      if scaling_mode == 'std':
        # print("Standard Scaling",'\n')
        if variance_feat == "var":
          with open(os.getcwd() + "/Scaler_var_spk/" + "std_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        elif not variance_feat:
          with open(os.getcwd() + "/Scaler_avg_ISI/" + "std_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        else:
          with open(os.getcwd() + "/Scaler_cov_spk/" + "std_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        
        x_test_scaled = scaler.transform(X_test)


      # Min-max Scaling
      elif scaling_mode == "minmax":
        # print("Minmax Scaling \n")
        if variance_feat == "var":
          with open(os.getcwd() + "/Scaler_var_spk/" + "minmax_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        elif not variance_feat:
          with open(os.getcwd() + "/Scaler_avg_ISI/" + "minmax_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)
        else:
          with open(os.getcwd() + "/Scaler_cov_spk/" + "minmax_Column_" + str(0) + ".pkl", 'rb') as file:  
            scaler = pickle.load(file)

        x_test_scaled = scaler.transform(X_test)


      if variance_feat == "var":
        if model_type == "Tree":
          with open(os.getcwd() + "/Models_var_spk/" + model_name + "_column_" + str(id) + ".pkl", 'rb') as file:  
            reg = pickle.load(file)
        else:
          if model_name == "ANN":
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_var_spk/" + model_name + "_column_" + str(id) +".h5", compile=False)
          else:
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_var_spk/" + model_name + "_column_" + str(id) +".h5", compile=False)

      elif not variance_feat:
        if model_type == "Tree":
          with open(os.getcwd() + "/Models_avg_ISI/" + model_name + "_column_" + str(id) + ".pkl", 'rb') as file:  
            reg = pickle.load(file)
        else:
          if model_name == "ANN":
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_avg_ISI/" + model_name + "_column_" + str(id) +".h5", compile=False)
          else:
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_avg_ISI/" + model_name + "_column_" + str(id) +".h5", compile=False)        

      else:
        if model_type == "Tree":
          with open(os.getcwd() + "/Models_var_spk/" + model_name + "_column_" + str(id) + ".pkl", 'rb') as file:  
            reg = pickle.load(file)
        else:
          if model_name == "ANN":
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_cov_spk/" + model_name + "_column_" + str(id) +".h5", compile=False)
          else:
            reg = tf.keras.models.load_model(os.getcwd() +  "/Models_cov_spk/" + model_name + "_column_" + str(id) +".h5", compile=False)


    
      if model_name == "CNN":
        test_pred = reg.predict(x_test_scaled.reshape((x_test_scaled.shape[0],row, col, chn)), verbose=0).flatten()
      elif model_name == "ANN":
        test_pred = reg.predict(x_test_scaled, verbose=0).flatten()
      else:
        test_pred = reg.predict(x_test_scaled)

      margin = optimal_maes[models_list.index(model_name)][id][-1] 
        
      y_t, y_p = getBinary(y_test, test_pred, margin=margin)

      detectecd = (sum(y_t!= y_p))/(len(y_t))
      prediction_wrt_column.append(detectecd)

    print(prediction_wrt_column)
    print(np.mean(prediction_wrt_column))
    print()

In [ ]:
# Loading MAEs_ISI to numpy array
with open('MAEs_ISI.npy', 'rb') as f:
    MAEs_ISI = np.load(f)

MAEs_ISI.shape

(4, 128, 3)

In [ ]:
# Trained models list
models_list = ["LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
nG = 128
ip_shape = (16,16,1)
scaling_mode = "std"
fol_Type = "Output_Missing"
crossbarName = "128x128"
model_type = "NN"
model_name = models_list[-2]


getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_ISI, models_list, variance_feat =False)

************************* Missing Spikes *************************
************************* Utilization : 10% *************************

Scaling Mode :  std
Model Name :  ANN 

[0.6582, 0.639, 0.7034, 0.7472, 0.848, 0.6782, 0.7252, 0.6978, 0.6802, 0.8196, 0.7824, 0.69, 0.832, 0.6254, 0.7468, 0.7412, 0.7678, 0.6246, 0.6358, 0.6438, 0.6264, 0.664, 0.7124, 0.672, 0.7342, 0.6856, 0.6496, 0.6446, 0.6446, 0.6582, 0.603, 0.7298, 0.7508, 0.6232, 0.6602, 0.6476, 0.6712, 0.6686, 0.7008, 0.677, 0.6886, 0.8338, 0.7826, 0.5504, 0.758, 0.8516, 0.6412, 0.7572, 0.708, 0.7806, 0.8002, 0.786, 0.7826, 0.7512, 0.7234, 0.7782, 0.7106, 0.81, 0.8196, 0.6538, 0.7252, 0.7268, 0.5426, 0.566, 0.8502, 0.7074, 0.9438, 0.8092, 0.6176, 0.7666, 0.5818, 0.557, 0.7232, 0.7306, 0.7802, 0.6806, 0.7182, 0.756, 0.6484, 0.738, 0.8188, 0.7148, 0.7822, 0.7274, 0.7424, 0.7972, 0.7868, 0.8746, 0.7294, 0.6998, 0.7138, 0.6486, 0.786, 0.7068, 0.545, 0.641, 0.839, 0.8972, 0.6622, 0.9574, 0.8496, 0.621, 0.8366, 0.804, 0.6222, 0.639

In [ ]:
# Trained models list
models_list = ["LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
nG = 128
ip_shape = (16,16,1)
scaling_mode = "std"
fol_Type = "Output_Spurious"
crossbarName = "128x128"
model_type = "NN"
model_name = models_list[-2]


getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_ISI, models_list, variance_feat =False)

************************* Spurious Spikes *************************
************************* Utilization : 10% *************************

Scaling Mode :  std
Model Name :  ANN 

[0.8982, 0.89, 0.8624, 0.8336, 0.7648, 0.8698, 0.8124, 0.849, 0.8714, 0.7678, 0.8066, 0.8704, 0.765, 0.88, 0.835, 0.8456, 0.8102, 0.9076, 0.904, 0.879, 0.9036, 0.8884, 0.8592, 0.8844, 0.843, 0.8676, 0.8952, 0.8956, 0.9046, 0.8778, 0.92, 0.823, 0.8192, 0.8818, 0.8794, 0.905, 0.8758, 0.889, 0.8554, 0.8734, 0.8778, 0.7522, 0.7836, 0.932, 0.8196, 0.737, 0.9032, 0.7876, 0.8412, 0.7812, 0.77, 0.7628, 0.7978, 0.8156, 0.844, 0.8134, 0.8484, 0.7844, 0.783, 0.8668, 0.84, 0.8472, 0.9534, 0.9512, 0.7114, 0.8214, 0.565, 0.771, 0.9182, 0.7908, 0.927, 0.9564, 0.8358, 0.8316, 0.801, 0.8798, 0.8338, 0.809, 0.8996, 0.7316, 0.7672, 0.8524, 0.7908, 0.823, 0.8184, 0.7914, 0.7672, 0.6904, 0.8072, 0.845, 0.8304, 0.8652, 0.7858, 0.8344, 0.9524, 0.8926, 0.7262, 0.6776, 0.8798, 0.5926, 0.6928, 0.8914, 0.7294, 0.7628, 0.8888, 0.8634, 0.

In [ ]:
# # Loading MAEs to numpy array
# with open('MAEs_COV.npy', 'rb') as f:
#     MAEs_COV = np.load(f)
# MAEs_COV.shape

In [ ]:
# models_list =  ["LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
# scaling_mode = "minmax"
# fol_Type = "Output_Spurious"
# model_type = "Tree"
# model_name = models_list[0]

# getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_COV, models_list, variance_feat =True)

In [ ]:
# models_list =  ["LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
# scaling_mode = "minmax"
# fol_Type = "Output_Missing"
# model_type = "Tree"
# model_name = models_list[0]

# getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_COV, models_list, variance_feat =True)

In [ ]:
# # Loading MAEs_VAR to numpy array
# with open('MAEs_VAR.npy', 'rb') as f:
#     MAEs_VAR = np.load(f)
# MAEs_VAR.shape

In [ ]:
# models_list =  ["LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
# scaling_mode = "minmax"
# nG = 128
# crossbarName = "128x128"
# ip_shape = (16,16,1)
# fol_Type = "Output_Spurious"
# model_type = "Tree"
# model_name = models_list[0]

# getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_VAR, models_list, variance_feat ="var")

In [ ]:
# models_list =  ["LGBMRegressor", "CatBoostRegressor", "ANN", "CNN"]
# scaling_mode = "minmax"
# nG = 128
# crossbarName = "128x128"
# ip_shape = (16,16,1)
# fol_Type = "Output_Missing"
# model_type = "Tree"
# model_name = models_list[0]

# getPrediction(nG, ip_shape, scaling_mode, fol_Type, crossbarName, model_type, model_name, MAEs_VAR, models_list, variance_feat ="var")